In [23]:
import os
import json
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pandas as pd
import pandas as pd
from difflib import SequenceMatcher
from sklearn.metrics import f1_score, precision_score, recall_score

In [24]:
model_version = 11 # 5 or 11
output_csv_names = ['paddle_ocr', 'easy_ocr', 'keras_ocr', 'transformer_ocr']

In [25]:
project_dir = os.path.dirname(os.getcwd())
dataset_location = os.path.join(project_dir, 'data', 'ocr_evaluation')
output_location = os.path.join(project_dir, 'data', f'yolo_v{model_version}')

In [26]:
df_true_location = f'{dataset_location}/output.csv'
df_true = pd.read_csv(df_true_location)
df_true['picture_name'] = df_true['picture_name'].apply(lambda x: 'plate_'+x)
df_true.set_index('picture_name', inplace=True)
df_true.head()

,text
picture_name,
plate_00808.jpg,046TS
plate_01433.jpg,912FB
plate_01063.jpg,044FF
plate_01576.jpg,M496T
plate_00263.jpg,155DW


In [27]:
def print_metrics(df_pred_name):
    df_pred_location = f'{output_location}/{df_pred_name}_pred.csv'
    df_pred = pd.read_csv(df_pred_location)
    df_pred.set_index('picture_name', inplace=True)
    
    # Ensure df_true is defined and has the same index
    # Assuming df_true is defined elsewhere and set as index 'picture_name'
    
    # Merging dataframes on 'picture_name' index to only evaluate overlapping entries
    df_merged = df_true.join(df_pred, lsuffix='_true', rsuffix='_pred', how='inner')

    # 1. Accuracy (Exact Match)
    exact_matches = (df_merged['text_true'] == df_merged['text_pred']).sum()
    accuracy = exact_matches / len(df_merged)

    # 2. Levenshtein Distance (Edit Distance) - Using SequenceMatcher
    def levenshtein_ratio(str1, str2):
        return SequenceMatcher(None, str(str1), str(str2)).ratio()

    # Handle NaN values by converting them to empty strings
    df_merged['levenshtein_ratio'] = df_merged.apply(
        lambda row: levenshtein_ratio(
            str(row.get('text_true', '')), 
            str(row.get('text_pred', ''))
        ), 
        axis=1
    )
    average_levenshtein_ratio = df_merged['levenshtein_ratio'].mean()

    # 3. Character-Level Precision, Recall, F1 Score
    def calculate_char_level_scores(true_text, pred_text):
        # Handle NaN values
        true_text = str(true_text)
        pred_text = str(pred_text)
        
        # Creating sets of characters for precision, recall, F1
        true_chars = set(true_text)
        pred_chars = set(pred_text)
        
        true_positive = len(true_chars & pred_chars)
        precision = true_positive / len(pred_chars) if len(pred_chars) > 0 else 0
        recall = true_positive / len(true_chars) if len(true_chars) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        return precision, recall, f1

    # Calculate character-level scores for each entry
    df_merged[['char_precision', 'char_recall', 'char_f1']] = df_merged.apply(
        lambda row: pd.Series(calculate_char_level_scores(row['text_true'], row['text_pred'])), 
        axis=1
    )

    # Aggregate the results
    average_precision = df_merged['char_precision'].mean()
    average_recall = df_merged['char_recall'].mean()
    average_f1_score = df_merged['char_f1'].mean()

    # Output results
    print(f'Results of {df_pred_name} model')
    print('---------------------------------')
    print(f"Accuracy (Exact Match): {accuracy:.2f}")
    print(f"Average Levenshtein Ratio: {average_levenshtein_ratio:.2f}")
    print(f"Character-Level Precision: {average_precision:.2f}")
    print(f"Character-Level Recall: {average_recall:.2f}")
    print(f"Character-Level F1 Score: {average_f1_score:.2f}")
    print('\n')

    df_merged_location = f'{output_location}/merged_{df_pred_name}.csv'
    df_merged.to_csv(df_merged_location)

In [28]:
for name in output_csv_names:
    print_metrics(name)

Results of paddle_ocr model
---------------------------------
Accuracy (Exact Match): 0.32
Average Levenshtein Ratio: 0.70
Character-Level Precision: 0.83
Character-Level Recall: 0.84
Character-Level F1 Score: 0.82


Results of easy_ocr model
---------------------------------
Accuracy (Exact Match): 0.08
Average Levenshtein Ratio: 0.41
Character-Level Precision: 0.52
Character-Level Recall: 0.42
Character-Level F1 Score: 0.44


Results of keras_ocr model
---------------------------------
Accuracy (Exact Match): 0.03
Average Levenshtein Ratio: 0.46
Character-Level Precision: 0.63
Character-Level Recall: 0.71
Character-Level F1 Score: 0.66


Results of transformer_ocr model
---------------------------------
Accuracy (Exact Match): 0.00
Average Levenshtein Ratio: 0.43
Character-Level Precision: 0.41
Character-Level Recall: 0.55
Character-Level F1 Score: 0.46


